In [1]:
from typing import List, Optional
from pydantic import BaseModel

class BoundingBox(BaseModel):
    """Bounding Box value type holds sizes in all three dimensions (in mm).
    These sizes indicate the maximum size that an entity takes in each of the three dimensions.
    """

    length: int
    width: int
    height: int

    @property
    def base_area(self):
        return self.length * self.width
    
    @property
    def volume(self):
        return self.length * self.width * self.height


class Offset(BaseModel):
    """Offset from a given origin (0,0,0) in mm."""

    x: int = 0
    y: int = 0
    z: int = 0


class Constraints(BaseModel):
    """Hold Bounding Boxes for Available Working Region, along with offsets that may further shrink this volume.
    Available Working Region is the region available for placement of modules.
    """

    working_region_bounding_box: BoundingBox
    pipette_bounding_box: BoundingBox
    pipette_offset: Offset


class Module(BaseModel):
    """Any module must provide a name and a bounding box."""
    name: str
    bounding_box: BoundingBox
    offset: Optional[Offset]


class Layout(BaseModel):
    """A layout model is the top most model for parsing and representing the contents of gero's layout file
    This contains both the constraints that limit layoutability, as well as bounding box and positional concerns of the modules that are to be provisioned in the layout.
    """

    constraints: Constraints
    modules: List[Module]


In [7]:
from json import load
from devtools import debug

file = "../sample_config_files/.gero/computed_layout.json"

with open(file=file) as f:
    data = load(f)
    # debug(data)

    layout = Layout(**data)

    # debug(layout.constraints)
    # debug(layout.modules)

    print(f"{len(layout.modules)} module(s) are to be added to the layout.")

3 module(s) are to be added to the layout.


In [8]:
wbox = layout.constraints.pipette_bounding_box
print(type(wbox))

print(f"area={wbox.base_area}mm^2, vol={wbox.volume}mm^3")

<class '__main__.BoundingBox'>
area=165750mm^2, vol=31492500mm^3
